In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sqlite3


## Loading data

In [3]:
con = sqlite3.connect('final.sqlite')

In [4]:
data = pd.read_sql_query("""Select * from Reviews where score != 3""", con)
data.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleanedtext,cleanedsummary
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witty little book makes son laugh loud recite ...,every book educational
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew reading sendak books watching really rosi...,love book miss hard cover version
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,fun way children learn months year learn poems...,chicken soup rice months
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,1,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,great little book read aloud nice rhythm well ...,good swingy rhythm reading aloud
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,1,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,book poetry months year goes month cute little...,great way learn months


In [5]:
data.shape

(364171, 13)

In [6]:
data['Score'].value_counts()

1    307061
0     57110
Name: Score, dtype: int64

In [7]:
# take 50000 each positive and negative random point from the data and concat
# the positive and negative in a single dataframe
positive_data = data[data['Score'] == 1].sample(n=50000)
negative_data = data[data['Score'] == 0].sample(n=50000)

# concat the both
final_data = pd.concat([positive_data, negative_data])
final_data.shape

(100000, 13)

In [8]:
#  change the time format unit
final_data['Time'] = pd.to_datetime(final_data['Time'], unit = 's')

# sort the data according to time
final_data = final_data.sort_values(by = 'Time')

final_data.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,cleanedtext,cleanedsummary
308,346041,374343,B00004CI84,A1B2IZU1JLZA6,Wes,19,23,0,2000-01-19,WARNING: CLAMSHELL EDITION IS EDITED TV VERSION,"I, myself always enjoyed this movie, it's very...",always enjoyed movie funny entertaining hesita...,warning clamshell edition edited tv version
425,417901,451923,B00004CXX9,ANIMV3SPDD8SH,Guy De Federicis,1,12,0,2001-06-11,CASPER IS THE GHOST WITH THE MOST,Michael Keaton brings no distinguishing charac...,michael keaton brings distinguishing character...,casper ghost
342,346037,374339,B00004CI84,AZRJH4JFB59VC,Lynwood E. Hines,21,23,0,2001-08-08,"Great movie, terrible DVD",I am continually amazed at the shoddy treatmen...,continually amazed shoddy treatment movies get...,great movie terrible dvd
270,346140,374449,B00004CI84,A3K3YJWV0N54ZO,Joey,2,3,1,2001-09-24,"Beetlejuice is a greatmovie, but they cheated ...",Just to warn you. when in trying to trick you ...,warn trying trick widescreen format company ma...,beetlejuice greatmovie cheated dvd
311,346031,374333,B00004CI84,A1CZICCYP2M5PX,Christian Pelchat,0,0,1,2002-02-07,An entertaining odd fantasy comedy.,When Two Couple (Geena Davis & Alec Baldwin) d...,two couple geena davis alec baldwin died car a...,entertaining odd fantasy comedy


## Split the data into train and test

In [9]:
X = final_data['cleanedtext']
y = final_data['Score']
print('Shape of X : ',X.shape)
print('Shape of y : ',y.shape)

print()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)



Shape of X :  (100000,)
Shape of y :  (100000,)



### KNN using Bag of words